In [18]:
import pandas as pd
import joblib
from tensorflow.keras.models import load_model

# Memuat model yang telah disimpan
model = load_model('model_preferensi_tensorflow.h5')
label_encoder = joblib.load('label_encoder.pkl')
preprocessor = joblib.load('preprocessor.pkl')

# Memuat dataset tempat wisata Bali
df = pd.read_csv("dataset_tempat_wisata_bali.csv")

# Mengubah rating yang menggunakan koma menjadi titik dan memastikan tipe data float
df['rating'] = df['rating'].astype(str).str.replace(',', '.').astype(float)

# Menghapus baris dengan nilai kosong pada kolom 'rating'
df_clean = df.dropna(subset=['rating'])

# Menampilkan daftar tempat wisata
def tampilkan_list_wisata():
    print("\nDaftar tempat wisata yang ada:")
    for index, row in df_clean.iterrows():
        print(f"{row['nama']} - {row['kategori']} - {row['kabupaten_kota']} - Rating: {row['rating']}")

# Fungsi untuk memberikan rekomendasi nama tempat wisata
def rekomendasi_wisata(kategori, kabupaten_kota, rating_min):
    # Preprocessing input
    input_data = pd.DataFrame([[kategori, kabupaten_kota, rating_min]], columns=['kategori', 'kabupaten_kota', 'rating'])
    input_data_preprocessed = preprocessor.transform(input_data)

    # Prediksi preferensi menggunakan model
    preferensi_pred = model.predict(input_data_preprocessed)
    preferensi_class = preferensi_pred.argmax(axis=1)
    preferensi = label_encoder.inverse_transform(preferensi_class)

    # Filter dataset berdasarkan kategori, kabupaten/kota, dan rating
    rekomendasi = df_clean[(df_clean['kategori'].str.lower() == kategori.lower()) &
                           (df_clean['kabupaten_kota'].str.lower() == kabupaten_kota.lower()) &
                           (df_clean['rating'] >= rating_min) &
                           (df_clean['rating'] <= 5)]  # Rating maksimum 5

    # Mengambil nama tempat wisata dari hasil filter
    if len(rekomendasi) > 0:
        return rekomendasi[['nama', 'rating']].to_dict(orient='records')
    else:
        return ["Tidak ada tempat wisata yang sesuai dengan kriteria"]

# Fungsi untuk memilih kategori
def pilih_kategori():
    kategori_options = ['Alam', 'Umum', 'Sejarah', 'Budaya']
    print("\nPilih kategori wisata:")
    for idx, kategori in enumerate(kategori_options, 1):
        print(f"{idx}. {kategori}")
    pilihan_kategori = int(input("\nMasukkan pilihan kategori (1-4): "))
    return kategori_options[pilihan_kategori - 1]

# Fungsi untuk memilih kabupaten/kota
def pilih_kabupaten_kota():
    kabupaten_kota_options = ['Kabupaten Badung', 'Kabupaten Gianyar', 'Kabupaten Tabanan',
                              'Kabupaten Bangli', 'Kabupaten Klungkung', 'Kabupaten Karangasem',
                              'Kabupaten Buleleng', 'Kota Denpasar', 'Kota Singaraja']
    print("\nPilih kabupaten/kota wisata:")
    for idx, kota in enumerate(kabupaten_kota_options, 1):
        print(f"{idx}. {kota}")
    pilihan_kabupaten_kota = int(input("\nMasukkan pilihan kabupaten/kota (1-9): "))
    return kabupaten_kota_options[pilihan_kabupaten_kota - 1]

# Fungsi untuk memilih rating minimum
def pilih_rating_min():
    print("\nMasukkan rating minimum (antara 1 dan 5): ")
    rating_min = float(input("Rating minimum: "))
    return rating_min

# Menampilkan semua tempat wisata
tampilkan_list_wisata()

# Input pengguna dengan pilihan
kategori_input = pilih_kategori()
kabupaten_kota_input = pilih_kabupaten_kota()
rating_min_input = pilih_rating_min()

# Dapatkan rekomendasi nama tempat wisata
rekomendasi = rekomendasi_wisata(kategori_input, kabupaten_kota_input, rating_min_input)

# Menampilkan rekomendasi
if isinstance(rekomendasi, list) and rekomendasi[0] == "Tidak ada tempat wisata yang sesuai dengan kriteria":
    print(rekomendasi[0])
else:
    print("\nRekomendasi tempat wisata yang sesuai dengan kriteria:")
    for rekomendasi_item in rekomendasi:
        print(f"{rekomendasi_item['nama']} - Rating: {rekomendasi_item['rating']}")



Daftar tempat wisata yang ada:
Taman Mumbul Sangeh - Alam - Kabupaten Badung - Rating: 4.6
Sangeh Monkey Forest - Alam - Kabupaten Badung - Rating: 4.6
Objek Wisata Sangeh - Umum - Kabupaten Badung - Rating: 4.6
Satria Gatotkaca Park - Umum - Kabupaten Badung - Rating: 4.6
Desa Wisata Penarungan - Umum - Kabupaten Badung - Rating: 5.0
Waterblow - Umum - Kabupaten Badung - Rating: 4.4
Pantai Mengening - Rekreasi - Kabupaten Badung - Rating: 4.7
Kawasan Pariwisata Nusa Dua - Umum - Kabupaten Badung - Rating: 4.6
Gunung Payung Cultural Park - Budaya - Kabupaten Badung - Rating: 4.6
Taman Anyar - Traditional Balinese House - Alam - Kabupaten Badung - Rating: 4.7
Garuda Wisnu Kencana Cultural Park - Budaya - Kabupaten Badung - Rating: 4.5
Desa Wisata Munggu - Umum - Kabupaten Badung - Rating: 4.4
Taman Ayun Temple - Budaya - Kabupaten Badung - Rating: 4.6
Pantai Double - Six - Rekreasi - Kabupaten Badung - Rating: 4.7
Tanah Wuk - Umum - Kabupaten Badung - Rating: 4.1
Taman Sari Wisata Baha